# Progetto BigData
## Valerio Di Zio e Francesco Magnani

intro here

In [1]:
%%configure -f
{"executorMemory":"8G", "numExecutors":2, "executorCores":3, "conf": {"spark.dynamicAllocation.enabled": "false"}}

In [2]:
val bucketname = "unibo-bd2324-vdizio"

val path_book_sample = "s3a://"+bucketname+"/project/book_sample134.json"
val path_book_sample32row = "s3a://"+bucketname+"/project/book_sample32row.json"
val path_positive_words = "s3a://"+bucketname+"/project/positive-words.txt"
val path_negative_words = "s3a://"+bucketname+"/project/negative-words.txt"

"SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/" + sc.applicationId + "/"

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
0,application_1707745135143_0001,spark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

bucketname: String = unibo-bd2324-vdizio
path_book_sample: String = s3a://unibo-bd2324-vdizio/project/book_sample134.json
path_book_sample32row: String = s3a://unibo-bd2324-vdizio/project/book_sample32row.json
path_positive_words: String = s3a://unibo-bd2324-vdizio/project/positive-words.txt
path_negative_words: String = s3a://unibo-bd2324-vdizio/project/negative-words.txt
res3: String = SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/application_1707745135143_0001/


In [97]:
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.DataFrame
import org.apache.spark.broadcast.Broadcast
import org.apache.spark.HashPartitioner

val partitioner = new HashPartitioner(40)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import org.apache.spark.rdd.RDD
import org.apache.spark.sql.DataFrame
import org.apache.spark.broadcast.Broadcast
import org.apache.spark.HashPartitioner
partitioner: org.apache.spark.HashPartitioner = org.apache.spark.HashPartitioner@28


In [4]:
case class BookReview(
    id: String,
    overall:Double,
    reviewTime:String,
    reviewerID:String,
    asin:String,
    reviewText:String,
    summary:String
)

object BookReview{
    def extract(row:org.apache.spark.sql.Row) = {
        val overall = row.getAs[Double]("overall")
        val reviewTime = row.getAs[String]("reviewTime")
        val reviewerID = row.getAs[String]("reviewerID")
        val asin = row.getAs[String]("asin")
        val reviewText = row.getAs[String]("reviewText")
        val summary = row.getAs[String]("summary")
        val id = reviewerID + "-" + asin
    
        new BookReview(id, overall, reviewTime, reviewerID, asin, reviewText, summary)
    }
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

defined class BookReview
defined object BookReview
Companions must be defined together; you may wish to use :paste mode for this.


In [182]:
val bookSampleRdd32row = spark.read.option("mode", "DROPMALFORMED").json(path_book_sample).map(BookReview.extract)
//val bookSampleRdd32row = spark.read.json(path_book_sample32row).map(BookReview.extract)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

bookSampleRdd32row: org.apache.spark.sql.Dataset[BookReview] = [id: string, overall: double ... 5 more fields]


In [183]:
val positiveWords = sc.textFile(path_positive_words)
val negativeWords = sc.textFile(path_negative_words)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

positiveWords: org.apache.spark.rdd.RDD[String] = s3a://unibo-bd2324-vdizio/project/positive-words.txt MapPartitionsRDD[1928] at textFile at <console>:42
negativeWords: org.apache.spark.rdd.RDD[String] = s3a://unibo-bd2324-vdizio/project/negative-words.txt MapPartitionsRDD[1930] at textFile at <console>:42


In [184]:
def splitAndClean(id:String, text: String): Array[(String, String, Int)] = 
    text.split("\\s+").map(el => (id, el.toLowerCase.replaceAll("[^a-zA-Z\\s]", ""), 1))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

splitAndClean: (id: String, text: String)Array[(String, String, Int)]


# Tentativi di ottimizzazione
1. utilizzo di variabili broadcast per le liste di parole uguali per tutti
2. ripartizione dei dataset per ottimizzare il join

In [185]:
def joinAndMapWords(review:DataFrame, wordsDataset: Broadcast[Array[String]]): RDD[(String, Int)] = {
    val df = review.sparkSession.createDataFrame(wordsDataset.value.map(Tuple1.apply)).toDF("word")
    val aggregRev = review.
        map (row => ((row.getAs[String]("id"), row.getAs[String]("word")), row.getAs[Int]("occurrencies"))).
        rdd.
        reduceByKey(_ + _).
        map{ case ((k1, k2), v1) => (k1, k2, v1)}
    
        val aggregRevDF = aggregRev.toDF("id", "word", "occurrencies")
    
        return aggregRevDF.
            join(df, aggregRevDF("word") === df("word")).
            map(row => (row.getAs[String]("id"), row.getAs[Int]("occurrencies"))).
            rdd.
            reduceByKey(_ + _)
}

    

val reviewsRdd = bookSampleRdd32row.
    select("id", "reviewText").
    dropDuplicates(Seq("id")).
    filter(review => review.getAs[String]("reviewText") != null && review.getAs[String]("reviewText").length > 3).
    flatMap(review => splitAndClean(review.getAs[String]("id"), review.getAs[String]("reviewText"))).
    filter(row => row._2.length > 3).
    toDF("id", "word", "occurrencies").
    cache()

val positiveWordsBroadcast = sc.broadcast(positiveWords.collect())
val negativeWordsBroadcast = sc.broadcast(negativeWords.collect())


val withPositiveRdd = joinAndMapWords(reviewsRdd, positiveWordsBroadcast)
    
val withNegativeRdd = joinAndMapWords(reviewsRdd, negativeWordsBroadcast)


val reducedResult = withPositiveRdd.
    fullOuterJoin(withNegativeRdd).
     map ({ case (key, (value1, value2)) => (key, (value1.getOrElse(0) - value2.getOrElse(0)))})

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

joinAndMapWords: (review: org.apache.spark.sql.DataFrame, wordsDataset: org.apache.spark.broadcast.Broadcast[Array[String]])org.apache.spark.rdd.RDD[(String, Int)]
reviewsRdd: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: string, word: string ... 1 more field]
positiveWordsBroadcast: org.apache.spark.broadcast.Broadcast[Array[String]] = Broadcast(620)
negativeWordsBroadcast: org.apache.spark.broadcast.Broadcast[Array[String]] = Broadcast(622)
withPositiveRdd: org.apache.spark.rdd.RDD[(String, Int)] = ShuffledRDD[1963] at reduceByKey at <console>:55
withNegativeRdd: org.apache.spark.rdd.RDD[(String, Int)] = ShuffledRDD[1982] at reduceByKey at <console>:55
reducedResult: org.apache.spark.rdd.RDD[(String, Int)] = MapPartitionsRDD[1986] at map at <console>:46


In [228]:
val resultDataframe = reducedResult.toDF("key", "value").cache()

val minValue = resultDataframe.agg(min("value")).head().getInt(0).toDouble
val maxValue = resultDataframe.agg(max("value")).head().getInt(0).toDouble


// Calculate mean and standard deviation
val meanAndStdDF = resultDataframe.agg(mean("value"), stddev("value")).collect()
val meanValue = meanAndStdDF(0)(0).asInstanceOf[Double]
val stdValue = meanAndStdDF(0)(1).asInstanceOf[Double]

// Normalize the data using Z-score normalization formula
// Normalize the data using Z-score normalization formula
val normalizedDF = resultDataframe.withColumn("z_score", (col("value") - meanValue) / stdValue)

// Scale the normalized values to be between -1 and 1
val minMaxDF = normalizedDF.agg(min("z_score"), max("z_score")).collect()
val minZScore = minMaxDF(0)(0).asInstanceOf[Double]
val maxZScore = minMaxDF(0)(1).asInstanceOf[Double]

val normalizedAndScaledDF = normalizedDF.withColumn("normalized_value", 
  (col("z_score") - minZScore) / (maxZScore - minZScore) * 4 + 1
).select("key", "normalized_value").rdd.map(row => (row.getAs[String](0), row.getAs[Double](1)))



// a = min, b = max, c = 1 e d = 5
def mapIntToOverallVote(toMap: Double, fromMin: Double, fromMax: Double, toMin:Double=1, toMax:Double=5): Double =
     toMin + ((toMax-toMin)/(fromMax-fromMin))*(toMap-fromMin)
    

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

resultDataframe: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [key: string, value: int]
minValue: Double = -82.0
maxValue: Double = 91.0
meanAndStdDF: Array[org.apache.spark.sql.Row] = Array([1.8820141631722722,3.570195472560125])
meanValue: Double = 1.8820141631722722
stdValue: Double = 3.570195472560125
normalizedDF: org.apache.spark.sql.DataFrame = [key: string, value: int ... 1 more field]
minMaxDF: Array[org.apache.spark.sql.Row] = Array([-23.495076056163935,24.96165448692443])
minZScore: Double = -23.495076056163935
maxZScore: Double = 24.96165448692443
normalizedAndScaledDF: org.apache.spark.rdd.RDD[(String, Double)] = MapPartitionsRDD[2742] at map at <console>:46
mapIntToOverallVote: (toMap: Double, fromMin: Double, fromMax: Double, toMin: Double, toMax: Double)Double


In [215]:
val normalizedResult = 
resultDataframe.
    map({row => (row.getAs[String](0), mapIntToOverallVote(row.getAs[Int](1).toDouble, minValue, maxValue))}).printSchema()



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- _1: string (nullable = true)
 |-- _2: double (nullable = false)

normalizedResult: Unit = ()


In [229]:
val relativeErrors = 
bookSampleRdd32row.
    map(review => (review.id, review.overall)).rdd.
    fullOuterJoin(normalizedAndScaledDF).
    map ({ case (key, (real, estimated)) => 
            (key, Math.abs(real.getOrElse[Double](3) - estimated.getOrElse[Double](3)) / real.getOrElse[Double](3) )
        })
        

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

relativeErrors: org.apache.spark.rdd.RDD[(String, Double)] = MapPartitionsRDD[2753] at map at <console>:46


In [230]:
val meanError = (relativeErrors.map(row => row._2).sum / relativeErrors.count)

println("Errore Medio: " + meanError*100 + "%")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

meanError: Double = 0.42554253489305555
Errore Medio: 42.55425348930555%
